# Baseline
## Import

In [1]:
# %pip install accelerate
# %pip install -i https://pypi.org/simple/ bitsandbytes
# %pip install transformers[torch] -U

# %pip install datasets
# %pip install langchain
# %pip install langchain_community
# %pip install PyMuPDF
# %pip install sentence-transformers
# %pip install faiss-gpu

In [1]:
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vector DB

In [3]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

## DB 생성

In [8]:
import os 

In [4]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
# pdf_databases = process_pdfs_from_dataframe(df, base_directory)

In [2]:
df = pd.read_csv('./train.csv')

In [5]:
df['Answer'][0]

'2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.'

## Model Import

In [21]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID 
    model_id = "google/gemma2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )
    
    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

# LLM 파이프라인
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 15/15 [00:11<00:00,  1.27it/s]
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


## 모델 학습

In [ ]:
df

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)   

os.environ["TOKENIZERS_PARALLELISM"] = "false"

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict['train'],
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 5,
        max_steps=3000,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        optim="paged_adamw_8bit",
        warmup_steps=300,
        learning_rate=2e-4,
        bf16=True, 
        logging_steps=100,
        report_to="wandb",
        push_to_hub=False,
    ),
    peft_config=lora_config,
    formatting_func=generate_prompts_train
)

## Langchain을 이용한 추론

In [22]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:08<14:28,  8.95s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,00,00원입니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:   2%|▏         | 2/98 [00:17<13:57,  8.73s/it]

...

Answer: 국가결산보고서는 국가회계기준에 따라 우발부채를 의무 이행을 위해 자원이 유출될 가능성이 희박하지 않은 한 주석5 ‘우발사항 및 약정사항’에 공시하고 있으며, 재정상태표에는 인식하지 않음12)​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:  99%|█████████▉| 97/98 [14:07<00:08,  8.84s/it]

Answer: 우발부채란 미래의 특정 사건(들)이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 그 이상의 조건이 충족되어야 금융거래로 인식된다는 점에서 ‘부채’와 차이6)​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions: 100%|██████████| 98/98 [14:16<00:00,  8.74s/it]

Answer: 보증이란 채무자가 채무의 이행에 관하여 채무자가 아닌 제3자(보증인)에게 채무의 이행을 청구할 수 있는 권리를 부여하는 것을 말하며, 보증은 채무자가 채무의 이행에 관하여 채무자가 아닌 제3자(보증인)에게 채무의 이행을 청구할 수 있는 권리를 부여하는 것을 말함. 보증은 채무자가 채무의 이행에 관하여 채무자가 아닌 제3자(보증인)에게 채무의 이행을 청구할 수 있는 권리를 부여하는 것을 말하며, 보증은 채무자가 채무의 이행에 관하여 채무자가 아닌 제3자(



## Submission

In [8]:
submit_df = pd.read_csv("./baseline_submission.csv")
submit_df

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,00,00원입니다.​​​​​..."
1,TEST_001,사업 목적​혁신창업사업화자금(융자) (5152-301)​1. 사업 코드 정보​구분​...
2,TEST_002,"중소기업창업지원법 제35조, 제36조, 제37조, 제38조, 제39조, 제40조, ..."
3,TEST_003,혁신창업사업화자금은 창업 후 3년 이내의 중소기업을 대상으로 혁신적인 기술과 아이디...
4,TEST_004,2020년 혁신창업사업화자금 신규 지원 내역은 다음과 같습니다.​혁신창업사업화자금(...
...,...,...
93,TEST_093,다음 정보를 바탕으로 질문에 답하세요:​​​​​​​​​​​​​​​​​​​​​​​​​...
94,TEST_094,미래사회보장급여에 대한 순의무란 미래사회보장급여에 대한 순의무란 미래사회보장급여에 ...
95,TEST_095,국가결산보고서는 국가회계기준에 따라 우발부채를 의무 이행을 위해 자원이 유출될 가능...
96,TEST_096,"우발부채란 미래의 특정 사건(들)이 일어나지 않는 한 발생하지 않는 의무로, 하나 ..."


In [23]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)


In [4]:
df = pd.read_csv("./baseline_submission.csv")
df

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,00,00원입니다.​​​​​..."
1,TEST_001,사업 목적​혁신창업사업화자금(융자) (5152-301)​1. 사업 코드 정보​구분​...
2,TEST_002,"중소기업창업지원법 제35조, 제36조, 제37조, 제38조, 제39조, 제40조, ..."
3,TEST_003,혁신창업사업화자금은 창업 후 3년 이내의 중소기업을 대상으로 혁신적인 기술과 아이디...
4,TEST_004,2020년 혁신창업사업화자금 신규 지원 내역은 다음과 같습니다.​혁신창업사업화자금(...
...,...,...
93,TEST_093,다음 정보를 바탕으로 질문에 답하세요:​​​​​​​​​​​​​​​​​​​​​​​​​...
94,TEST_094,미래사회보장급여에 대한 순의무란 미래사회보장급여에 대한 순의무란 미래사회보장급여에 ...
95,TEST_095,국가결산보고서는 국가회계기준에 따라 우발부채를 의무 이행을 위해 자원이 유출될 가능...
96,TEST_096,"우발부채란 미래의 특정 사건(들)이 일어나지 않는 한 발생하지 않는 의무로, 하나 ..."
